In [1]:
import psycopg2
from psycopg2 import extras
from psycopg2 import sql
from psycopg2.extensions import register_adapter, AsIs
import pandas as pd
from ast import literal_eval
import numpy

In [2]:
# Connection to server
def conectionDB(dbname, port, user, password, host):
    conn_string = "dbname={} port={} user={} password={} host={}".format(dbname, port, user, password, host)
    #print("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

In [ ]:
def add_data(conn, cursor, data, table):
    cursor.executemany( sql.SQL("insert into {} values ({} , {})")
        .format(sql.Identifier(table)), data)
    conn.commit()

In [ ]:
path='./datasets' 
#path_res='%s/p6_Corpus.csv' %(path)
path_res='%s/p6_Corpus_reducido.csv' % (path)

data = pd.read_csv(path_res, sep=",", header=0, converters={"Corpus": literal_eval})

In [ ]:
data1 = [[data.iloc[i,0], data.iloc[i,1]] for i in range(len(data.index))]
data1

## Populate a database with csv file

In [3]:
# Config
dbname = 'consumptionDB'
port = '5432'
user = 'consumptionuser'
password = 'consumptionuser2020'
host = '172.250.1.22'

In [ ]:
# Create a simple query
conn = conectionDB(dbname, port, user, password, host)
cursor = conn.cursor()
add_data(conn, cursor, data1)
conn.close()

In [4]:
conn = conectionDB(dbname, port, user, password, host)
cursor = conn.cursor()
query = """select * from consumption"""
df1 = pd.read_sql(query, conn)  # storing the query in a pandas dataframe
conn.close()
df1

,client,sequence
0,++5u+heOZ8o=,"[SA21, C20, SA49, C20, SA11]"
1,++70ByX0a3Q=,"[SA11, C20, SA10, C20, SA5, C20, SA21, C20, SA32]"
2,++7i5fi6kBU=,"[SA31, C20, SA51]"
3,++FMZ01PEY4=,"[SA48, C66, SA34]"
4,++FVgfEREuc=,"[SA44, C20, SA38]"
...,...,...
175113,zzr09cNFFtY=,"[SA9, C20, SA73]"
175114,zzvPsWkPQi8=,"[SA73, C20, SA9]"
175115,zzxOoziChuk=,"[SA31, C20, SA27, C20, SA23]"
175116,zzxjK1vzG4E=,"[SA31, C20, SA30, C20, SA20]"


In [ ]:
corpus = list(data["Corpus"])

In [ ]:
set_corpus = []
for s  in corpus:
    set_corpus += s
set_corpus = list(set(set_corpus))
set_corpus.sort()

In [ ]:
dict_corpus = { set_corpus[i] : i for i in range(0, len(set_corpus) ) }
dict_corpus

In [ ]:
data2 = set_corpus
data2

In [ ]:
#conn1 = conectionDB(dbname, port, user, password, host)
#cur1 = conn1.cursor()
#for d in data2:
#    cur1.execute("insert into itemset_consumption(itemset) values ('%s')" % d)
#    conn1.commit()
#conn1.close()

In [ ]:
#for i in range(len(data.index)):
#    add_data(cursor, [data.iloc[i,0], data.iloc[i,1]])
#    cursor.statusmessage
#cursor.execute(
#    sql.SQL("insert into {} values (%s, %s)")
#        .format(sql.Identifier('consumption')),
#    ['+++g8j9k+5A=', ['SA5', 'C20', 'SA77']])
#colnames = [desc[0] for desc in cursor.description]
#colnames

In [ ]:
#cursor.execute(
#    sql.SQL("delete from {}")
#       .format(sql.Identifier('consumption')),)
#cursor.statusmessage

In [ ]:
conn1 = conectionDB(dbname, port, user, password, host)
cur1 = conn1.cursor()
cur1.execute(
    sql.SQL("delete from {}")
       .format(sql.Identifier('consumption_translate')),)
cur1.statusmessage
conn1.commit()           
conn1.close()

In [ ]:
#conn1 = conectionDB(dbname, port, user, password, host)
#cur1 = conn1.cursor()
#cur1.execute(
#    sql.SQL("delete from {}")
#      .format(sql.Identifier('itemset_consumption')),)
#cur1.statusmessage
#conn1.commit()           
#conn1.close()

In [ ]:
#ALTER SEQUENCE itemset_consumption_itemset_index_seq RESTART WITH 1;

In [5]:
conn1 = conectionDB(dbname, port, user, password, host)
query = """select * from itemset_consumption"""
df2 = pd.read_sql(query, conn1)  # storing the query in a pandas dataframe
conn1.close()
df2

,itemset_index,itemset
0,5,C04
1,10,C09
2,13,C12
3,19,C18
4,20,C19
...,...,...
157,148,SA71
158,152,SA75
159,153,SA76
160,155,SA78


In [6]:
#df2.loc[df2["itemset"]=='SA21'].iloc[0,0]
rows = [] 
for seq in df1.iloc[0:,1]:
    row = []
    for item in seq:
        row.append(df2.loc[df2["itemset"]==item].iloc[0,0])
    rows.append(row)    

In [7]:
rows

[[93, 21, 123, 21, 82],
 [82, 21, 81, 21, 124, 21, 93, 21, 105],
 [104, 21, 126],
 [122, 67, 107],
 [118, 21, 111],
 [99, 67, 60, 162, 2, 81, 21, 113, 21, 86, 67, 83],
 [94, 45, 162, 45, 124, 21, 113, 29, 162, 21, 146, 45, 93],
 [134, 57, 95],
 [120, 8, 109],
 [88, 39, 119, 39, 90],
 [85, 79, 91, 21, 21, 91, 79, 80, 21, 113, 21, 146, 21, 120],
 [94, 21, 136],
 [131, 21, 82, 21, 85],
 [85, 21, 144],
 [116, 53, 53, 82, 53, 99],
 [84, 21, 145],
 [146, 35, 107, 35, 162, 35, 80, 35, 100],
 [88, 57, 109, 57, 100],
 [80,
  21,
  80,
  21,
  21,
  80,
  21,
  21,
  21,
  21,
  80,
  21,
  80,
  38,
  1,
  91,
  21,
  21,
  21,
  21,
  80,
  21,
  102,
  21,
  21,
  21,
  80,
  21,
  80,
  21,
  80,
  42,
  80,
  21,
  91,
  21,
  21,
  80,
  21,
  21,
  80,
  21,
  21,
  91,
  21,
  102,
  21,
  21,
  21,
  21,
  102,
  21,
  91,
  21,
  42,
  80,
  21,
  21,
  102,
  21,
  80,
  21,
  162],
 [96, 41, 133],
 [80, 21, 102, 21, 96, 21, 89, 21, 86, 57, 88],
 [82, 21, 148],
 [99, 21, 130],
 [136, 

In [8]:
data1 = [[df1.iloc[i,0], rows[i]] for i in range(len(df1.index))]
data1

[['++5u+heOZ8o=', [93, 21, 123, 21, 82]],
 ['++70ByX0a3Q=', [82, 21, 81, 21, 124, 21, 93, 21, 105]],
 ['++7i5fi6kBU=', [104, 21, 126]],
 ['++FMZ01PEY4=', [122, 67, 107]],
 ['++FVgfEREuc=', [118, 21, 111]],
 ['++L6AKuw+Ro=', [99, 67, 60, 162, 2, 81, 21, 113, 21, 86, 67, 83]],
 ['++Mr4ALD95c=', [94, 45, 162, 45, 124, 21, 113, 29, 162, 21, 146, 45, 93]],
 ['++R8GLgt6jo=', [134, 57, 95]],
 ['++S+MUM5TmU=', [120, 8, 109]],
 ['++T363m1xs4=', [88, 39, 119, 39, 90]],
 ['++TRf15NWl4=', [85, 79, 91, 21, 21, 91, 79, 80, 21, 113, 21, 146, 21, 120]],
 ['++Yg9SnovE4=', [94, 21, 136]],
 ['++clOnxbpk0=', [131, 21, 82, 21, 85]],
 ['++jQgeK5yQg=', [85, 21, 144]],
 ['++oISMACsR0=', [116, 53, 53, 82, 53, 99]],
 ['++prAEPvDk4=', [84, 21, 145]],
 ['++qQNIpeqpA=', [146, 35, 107, 35, 162, 35, 80, 35, 100]],
 ['++u2E3fTb7A=', [88, 57, 109, 57, 100]],
 ['++xDKRJ6IeY=',
  [80,
   21,
   80,
   21,
   21,
   80,
   21,
   21,
   21,
   21,
   80,
   21,
   80,
   38,
   1,
   91,
   21,
   21,
   21,
   21,
   80

In [9]:
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

In [10]:
register_adapter(numpy.int64, addapt_numpy_int64)
conn1 = conectionDB(dbname, port, user, password, host)
cur1 = conn1.cursor()

for d in data1:
    cur1.execute(sql.SQL("insert into {} values (%s, %s)")
        .format(sql.Identifier("consumption_translate")), d)
conn1.commit()
cur1.statusmessage      
conn1.close()

In [11]:
conn = conectionDB(dbname, port, user, password, host)
cursor = conn.cursor()
query = """select * from consumption_translate"""
df3 = pd.read_sql(query, conn)  # storing the query in a pandas dataframe
conn.close()
df3

,client,sequence
0,++70ByX0a3Q=,"[82, 21, 81, 21, 124, 21, 93, 21, 105]"
1,++R8GLgt6jo=,"[134, 57, 95]"
2,++Yg9SnovE4=,"[94, 21, 136]"
3,++jQgeK5yQg=,"[85, 21, 144]"
4,++prAEPvDk4=,"[84, 21, 145]"
...,...,...
175113,zymMS8finO8=,"[113, 55, 128, 55, 96]"
175114,zz9E0Yxt2HM=,"[95, 21, 134]"
175115,zzWK0G9qPas=,"[130, 8, 99]"
175116,zzr09cNFFtY=,"[162, 21, 150]"
